In [ ]:
!pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.1/677.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 9.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from pymongo import MongoClient

from google.colab import drive
drive.mount("/content/gdrive")
import warnings
warnings.filterwarnings(action='ignore')

access_cert_path = '/content/gdrive/MyDrive/SoftwareProject2/sources/X509-cert-823328410369210316.pem'

Mounted at /content/gdrive


In [ ]:
from pymongo.server_api import ServerApi
from pymongo import MongoClient

#uri = "mongodb+srv://cluster0.abc0jxe.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
uri = "mongodb+srv://cluster0.5u1gnfg.mongodb.net/?authSource=%24external&authMechanism=MONGODB-X509&retryWrites=true&w=majority"
client = MongoClient(uri,
                     tls=True,
                     tlsCertificateKeyFile=access_cert_path,
                     server_api=ServerApi('1'))
db = client['Bookshop']
collection = db['Sales']
collection.count_documents({})

262976

In [ ]:
data = list(collection.find())
sales_data = pd.DataFrame(data)

In [ ]:
sales_data.drop(columns=['_id'], inplace=True)
sales_data['Sale Date'] = pd.to_datetime(sales_data['Sale Date'])
sales_data.set_index('Sale Date', inplace=True)

# Group by ISBN and month, and count the number of sales
monthly_sales = sales_data.groupby([sales_data.index.to_period('M'), 'ISBN']).size().reset_index(name='Sales')

# Pivot the table to have dates as rows and books as columns, filling missing values with 0
sales_pivot = monthly_sales.pivot_table(index='Sale Date', columns='ISBN', values='Sales', fill_value=0)


In [ ]:
# Split the data into training and test sets
train_data = sales_pivot[np.logical_and(sales_pivot.index.year < 2023,np.logical_not(np.logical_and(sales_pivot.index.month==12, sales_pivot.index.year==2022)))]
test_data = sales_pivot[np.logical_or(np.logical_and(sales_pivot.index.month==12, sales_pivot.index.year==2022),sales_pivot.index.year==2023)]

print(train_data.shape)
print(test_data.shape)


(59, 93)
(13, 93)


In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
import numpy as np

def adf_test(timeseries):
    result = adfuller(timeseries, autolag='AIC')
    return result[1]

# Function to find the optimal SARIMA order
def optimize_sarima(parameters_list, seasonal_parameters_list, d, D, s, time_series):
    """
    Return dataframe with parameters and corresponding AIC

    parameters_list - list with (p, q, P, Q) tuples
    d, D - integration order, seasonal integration order
    s - length of season
    time_series - the observed time series
    """

    results = []

    for param in parameters_list:
        for param_seasonal in seasonal_parameters_list:
            try:
                model = SARIMAX(time_series,
                                order=(param[0], d, param[1]),
                                seasonal_order=(param_seasonal[0], D, param_seasonal[1], s),
                                enforce_stationarity=False,
                                enforce_invertibility=False,
                                disp=False)
                results.append([param, param_seasonal, model.fit(disp=False).aic])
            except:
                continue

    result_df = pd.DataFrame(results, columns=['param', 'param_seasonal', 'aic'])
    result_df = result_df.sort_values(by='aic', ascending=True).reset_index(drop=True)

    return result_df


isbn_to_forecast = sales_pivot.columns[0]
single_book_sales = train_data[isbn_to_forecast]

adf_p_value = adf_test(single_book_sales)

if adf_p_value > 0.05:
    d = 1
else:
    d = 0

# Set seasonal parameters
# Assuming a yearly seasonality for book sales
s = 12
D = 1 if d == 0 else 0  # If we already have differenced, no need for seasonal differencing

# Define p, q, P, Q ranges to test
p = range(0, 3)  # AR terms
q = range(0, 3)  # MA terms
P = range(0, 2)  # Seasonal AR terms
Q = range(0, 2)  # Seasonal MA terms

parameters = [(x[0], x[1]) for x in list(np.ndindex((len(p), len(q))))]
seasonal_parameters = [(x[0], x[1]) for x in list(np.ndindex((len(P), len(Q))))]

# Optimize SARIMA parameters
optimal_params_df = optimize_sarima(parameters, seasonal_parameters, d, D, s, single_book_sales)

optimal_params_df.head()


/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood op

,param,param_seasonal,aic
0,"(2, 0)","(1, 0)",176.234329
1,"(2, 2)","(1, 0)",177.302246
2,"(2, 1)","(1, 0)",177.952680
3,"(1, 1)","(1, 0)",179.487934
4,"(1, 2)","(1, 0)",181.422372


In [ ]:
min_value_row = optimal_params_df.loc[optimal_params_df['aic'].idxmin()]
optimal_p, optimal_q = min_value_row['param']
optimal_P, optimal_Q = min_value_row['param_seasonal']

best_sarima_model = SARIMAX(single_book_sales,
                            order=(optimal_p, d, optimal_q),
                            seasonal_order=(optimal_P, D, optimal_Q, s),
                            enforce_stationarity=False,
                            enforce_invertibility=False)
best_sarima_model_fit = best_sarima_model.fit(disp=False)
best_sarima_model_fit.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                            7176023   No. Observations:                   59
Model:             SARIMAX(2, 0, 0)x(1, 1, 0, 12)   Log Likelihood                 -84.117
Date:                            Thu, 30 Nov 2023   AIC                            176.234
Time:                                    06:38:13   BIC                            182.220
Sample:                                01-31-2018   HQIC                           178.248
                                     - 11-30-2022                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.3203      0.221      1.447      0.148      -0.113       0.754
ar.L2          0.3364      0.222      1.515      0.130      -0.099       0.771
ar.S.L12      -0.9004      0.215     -4.191      0.000      -1.321      -0.479
sigma2         9.5843      2.162      4.433      0.000       5.347      13.821
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):                 3.37
Prob(Q):                              0.91   Prob(JB):                         0.19
Heteroskedasticity (H):               5.59   Skew:                             0.71
Prob(H) (two-sided):                  0.01   Kurtosis:                         3.65
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
single_book_sales_test = test_data[isbn_to_forecast]
single_book_sales_test

Sale Date
2022-12     5
2023-01     5
2023-02     3
2023-03     7
2023-04     5
2023-05     7
2023-06     7
2023-07    15
2023-08    17
2023-09     9
2023-10     7
2023-11     7
2023-12     5
Freq: M, Name: 7176023, dtype: int64

In [ ]:
# Predictions for 2194
from sklearn.metrics import mean_squared_error
sarima_predictions = best_sarima_model_fit.forecast(len(single_book_sales_test))
sarima_mse = mean_squared_error(single_book_sales_test, sarima_predictions)

new_sarima_residuals = single_book_sales_test - sarima_predictions
new_sarima_residuals, sarima_predictions, sarima_mse


(Sale Date
 2022-12    -1.610487
 2023-01     0.280390
 2023-02    -3.486947
 2023-03     3.079882
 2023-04     1.943084
 2023-05     3.049278
 2023-06    -1.263958
 2023-07     3.471355
 2023-08    10.476800
 2023-09     2.350860
 2023-10     0.008538
 2023-11     1.456188
 2023-12    -1.171519
 Freq: M, dtype: float64,
 2022-12     6.610487
 2023-01     4.719610
 2023-02     6.486947
 2023-03     3.920118
 2023-04     3.056916
 2023-05     3.950722
 2023-06     8.263958
 2023-07    11.528645
 2023-08     6.523200
 2023-09     6.649140
 2023-10     6.991462
 2023-11     5.543812
 2023-12     6.171519
 Freq: M, Name: predicted_mean, dtype: float64,
 13.063170622448597)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np

residuals_data = new_sarima_residuals.values.reshape(-1, 1)

# Normalize the residuals data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_residuals = scaler.fit_transform(residuals_data)

n_input = 1
n_features = 1
batch_size = 1
epochs = 200

generator = TimeseriesGenerator(scaled_residuals, scaled_residuals, length=n_input, batch_size=batch_size)

In [ ]:
# Define LSTM model
lstm_model = Sequential()
lstm_model.add(LSTM(50, activation='relu', input_shape=(n_input, n_features), return_sequences=True))
lstm_model.add(Dropout(0.2))
lstm_model.add(LSTM(50, activation='relu', return_sequences=False))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(25, activation='relu'))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 1, 50)             10400     
                                                                 
 dropout (Dropout)           (None, 1, 50)             0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense (Dense)               (None, 25)                1275      
                                                                 
 dropout_2 (Dropout)         (None, 25)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 2

In [ ]:

lstm_model.fit(generator, epochs=epochs, verbose=0)

lstm_predictions_scaled = lstm_model.predict(generator)

lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)

12/12 [==============================] - 1s 5ms/step


In [ ]:
mean_squared_error(single_book_sales_test.to_numpy()[1:], sarima_predictions.to_numpy()[1:] + lstm_predictions.ravel())

10.906826981664283

In [ ]:
lstm_predictions

array([[1.3615942],
       [1.4907986],
       [1.2453622],
       [1.7354045],
       [1.6332126],
       [1.7326063],
       [1.3842171],
       [1.7713617],
       [2.4689748],
       [1.6693887],
       [1.4713789],
       [1.5904247]], dtype=float32)

In [ ]:
import math

final_predictions = np.array([math.floor(x) for x in sarima_predictions.to_numpy()[1:] + lstm_predictions.ravel()])
final_predictions

array([ 6,  7,  5,  4,  5,  9, 12,  8,  9,  8,  7,  7])

In [ ]:
dates = pd.date_range(start='2024-01-01', end='2024-12-01', freq='MS')
time_series_df = pd.DataFrame(final_predictions, index=dates, columns=['Value'])

print(f"ISBN: {isbn_to_forecast}\n")
print(f"Forecast: \n{time_series_df}")

ISBN: 7176023

Forecast: 
            Value
2024-01-01      6
2024-02-01      7
2024-03-01      5
2024-04-01      4
2024-05-01      5
2024-06-01      9
2024-07-01     12
2024-08-01      8
2024-09-01      9
2024-10-01      8
2024-11-01      7
2024-12-01      7


In [ ]:
collection = db['SalesForecast']

#delete existing forecasts for the isbn
#to further update to only delete affected isbn and period combi
query = {"ISBN": isbn_to_forecast}
collection.delete_many(query)


forecast_df = pd.DataFrame(dates, columns=['Period'])
forecast_df['ISBN'] = isbn_to_forecast
forecast_df['Value'] = final_predictions
recordsSalesForecast = forecast_df.to_dict('records')
collection.insert_many(recordsSalesForecast)

InsertManyResult([ObjectId('656831c6273f3a5799f7a6fb'), ObjectId('656831c6273f3a5799f7a6fc'), ObjectId('656831c6273f3a5799f7a6fd'), ObjectId('656831c6273f3a5799f7a6fe'), ObjectId('656831c6273f3a5799f7a6ff'), ObjectId('656831c6273f3a5799f7a700'), ObjectId('656831c6273f3a5799f7a701'), ObjectId('656831c6273f3a5799f7a702'), ObjectId('656831c6273f3a5799f7a703'), ObjectId('656831c6273f3a5799f7a704'), ObjectId('656831c6273f3a5799f7a705'), ObjectId('656831c6273f3a5799f7a706')], acknowledged=True)

In [ ]:
doc_count = collection.count_documents({})
doc_count

12